In [1]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig, TrainingArguments
from trl import SFTTrainer

# !pip install -qU auto-gptq optimum


In [2]:
MODEL_ID = "TheBloke/zephyr-7B-alpha-GPTQ"
DATASET_ID = "bitext/Bitext-customer-support-llm-chatbot-training-dataset"
CONTEXT_FIELD= ""
INSTRUCTION_FIELD = "instruction"
TARGET_FIELD = "response"
BITS = 4
DISABLE_EXLLAMA = True
DEVICE_MAP = "auto"
USE_CACHE = False
LORA_R = 16
LORA_ALPHA = 16
LORA_DROPOUT = 0.05
BIAS = "none"
TARGET_MODULES = ["q_proj", "v_proj"]
TASK_TYPE = "CAUSAL_LM"
OUTPUT_DIR = "zephyr-support-chatbot"
BATCH_SIZE = 8
GRAD_ACCUMULATION_STEPS = 1
OPTIMIZER = "paged_adamw_32bit"
LR = 2e-4
LR_SCHEDULER = "cosine"
LOGGING_STEPS = 50
SAVE_STRATEGY = "epoch"
NUM_TRAIN_EPOCHS = 1
MAX_STEPS = 250
FP16 = True
PUSH_TO_HUB = True
DATASET_TEXT_FIELD = "text"
MAX_SEQ_LENGTH = 1024
PACKING = False


In [3]:
tokenizer = AutoTokenizer.from_pretrained("TheBloke/zephyr-7B-alpha-GPTQ")
tokenizer.pad_token = tokenizer.eos_token
bnb_config = GPTQConfig(bits=4,
                        disable_exllama=True,
                        device_map="auto",
                        use_cache=False,
                        lora_r=16,
                        lora_alpha=16,
                        tokenizer=tokenizer
                                )
#
model = AutoModelForCausalLM.from_pretrained("TheBloke/zephyr-7B-alpha-GPTQ",
                                              quantization_config=bnb_config,
                                              device_map="auto",
                                              use_cache=False,
                                              )

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.
You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute and has already quantized weights. However, loading attributes (e.g. ['use_cuda_fp16', 'use_exllama', 'max_input_length', 'exllama_config', 'disable_exllama']) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.


model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [8]:
sample_ocr="""Fast
NEW
Dissolution
Quick
Cleaning
सफेद
डिटरजेंट पाउडर
High Foam
515
Soft
Manufacturerkeled by:
Shantinath et argents Pvt Lt
by, rehcharacters the code embossed on
for unique blend
89360224901207
CUSTOMER CARE
3 2248 9408
the box fo low O. A) 16 Road, Kolkata 700058,
BF BT Row Godown
Kolkata 700058, C) Plot
60 Bahmapudustrial Park, Village - Sila,
a Guwa Ask
on Hands
terge.com OTHER
Generation
#MRP. ?
BA)08/22 130/-/65/Kg)
Net Wt.
2 kg
Pkd.
Safed with its
next-generation SmartZymes Technology,
provides Smart Care & Enduring Freshness
for your clothes.
Enduring
Freshness
Long-lasting
Refreshing
Fragrance.
Smart
vmes
Instructions For Use
For hand wash
1 scoop
For machine wash
2 scoops
Smart Care
Remarkably Clean
& Vibrant Clothes
scoop not included. 

   
 Smart Care & Enduring freshness,Long Lasting Refreshing FRAGRANCE,Remarkably CLEAN & VIBRANT clothes,"""

In [22]:
from peft import AutoPeftModelForCausalLM
from transformers import GenerationConfig
from transformers import AutoTokenizer
import torch

def process_data_sample(sample_ocr):

    processed_example = """<|system|>\n ###Instruction:
    You are an expert in extracting product claims from Product OCR input of the product image, please consider the below points before extracting those claims 
    Consider the below conditions before extracting the claims: 
        Condition 1 - Extract only the English claims that the product makes and please do not provide any explanation
        Condition 2 - Consider tests and certifications mentioned as claims
        Condition 3 - Do not consider quantity, expiry or directions to use as claims
        Condition 4 - Do not consider any nutritional information as claims
        Condition 5 - Do not consider contact, address or website information
        Condition 6 - If you do not find any product claims return None
        Condition 7 - Ignore other language claims
        Condition 9 - Please do not consider volume information as claim and also please do not consider instructions provided as claims 
        Condition 8 - Please do not provide any explanation, the extracted claims should be separated by comas for example like this - claim 1,claim 2,....claim n\n<|user|>\n""" + f"""{sample_ocr} \n ########################################\n\n Given the Product OCR input above, please extract the claims of the product """ + "\n<|assistant|>\n"

    return processed_example

# tokenizer = AutoTokenizer.from_pretrained("/content/zephyr-support-chatbot")

def get_response(sample_ocr):
    inp_str = process_data_sample(sample_ocr)

    inputs = tokenizer(inp_str, return_tensors="pt").to("cuda")
    generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.1,
    max_new_tokens=256,
    pad_token_id=tokenizer.eos_token_id
    )
    outputs = model.generate(**inputs, generation_config=generation_config)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [10]:
generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.1,
    max_new_tokens=256,
    pad_token_id=tokenizer.eos_token_id
)

In [11]:
# import time
# st_time = time.time()
# print(time.time()-st_time)

<|system|>
 ###Instruction:
    You are an expert in extracting product claims from Product OCR input of the product image, please consider the below points before extracting those claims 
    Consider the below conditions before extracting the claims: 
        Condition 1 - Extract only the English claims that the product makes and please do not provide any explanation
        Condition 2 - Consider tests and certifications mentioned as claims
        Condition 3 - Do not consider quantity, expiry or directions to use as claims
        Condition 4 - Do not consider any nutritional information as claims
        Condition 5 - Do not consider contact, address or website information
        Condition 6 - If you do not find any product claims return None
        Condition 7 - Ignore other language claims
        Condition 9 - Please do not consider volume information as claim and also please do not consider instructions provided as claims 
        Condition 8 - Please do not provide any ex

In [12]:
import pandas as pd


train=pd.read_csv("/LLM_data/claim_train_llm.csv")
test=pd.read_csv("/LLM_data/claim_test_llm.csv")

In [13]:
df=train.copy()

In [14]:
text_to_add="""###Instruction:
    You are an expert in extracting product claims from Product OCR input of the product image, please consider the below points before extracting those claims 
    Consider the below conditions before extracting the claims: 
        Condition 1 - Extract only the English claims that the product makes and please do not provide any explanation
        Condition 2 - Consider tests and certifications mentioned as claims
        Condition 3 - Do not consider quantity, expiry or directions to use as claims
        Condition 4 - Do not consider any nutritional information as claims
        Condition 5 - Do not consider contact, address or website information
        Condition 6 - If you do not find any product claims return None
        Condition 7 - Ignore other language claims
        Condition 9 - Please do not consider volume information as claim and also please do not consider instructions provided as claims 
        Condition 8 - Please do not provide any explanation, the extracted claims should be separated by comas for example like this - claim 1,claim 2,....claim n

### Product OCR input:"""
text_to_add

'###Instruction:\n    You are an expert in extracting product claims from Product OCR input of the product image, please consider the below points before extracting those claims \n    Consider the below conditions before extracting the claims: \n        Condition 1 - Extract only the English claims that the product makes and please do not provide any explanation\n        Condition 2 - Consider tests and certifications mentioned as claims\n        Condition 3 - Do not consider quantity, expiry or directions to use as claims\n        Condition 4 - Do not consider any nutritional information as claims\n        Condition 5 - Do not consider contact, address or website information\n        Condition 6 - If you do not find any product claims return None\n        Condition 7 - Ignore other language claims\n        Condition 9 - Please do not consider volume information as claim and also please do not consider instructions provided as claims \n        Condition 8 - Please do not provide any ex

In [15]:
df.head(2)

,id,Image_side,claim_0,claim_1,claim_2,claim_3,claim_4,claim_5,claim_6,claim_7,...,claim_13,claim_14,claim_15,claim_16,claim_17,claim_18,claim_19,claim_20,ocr_info,prompt_cliam
0,10305744_0,Front,REMOVE TOUGH STAINS & DIRT,NATURAL FRAGRANCE,FULL DISSOLUTION,PLANT BASED,CHILD SAFE,COLOUR SAFE,NON TOXIC,SAFE ON HANDS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LIQUID\nDETERGENT\nPREMIUM\nISO\n9001:29 15\nQ...,"REMOVE TOUGH STAINS & DIRT, NATURAL FRAGRANCE..."
1,10305744_1,Back,"HARD ON DIRT AND TOUGH STAINS, MAKES YOUR CLO...",SUITABLE FOR All TYPE of CLOTHS,OKTAA active detergent LIQUID IS SUITABLE FOR...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Net. Volume: 1000 ml\nM.R.P.3:\n355.00\nLDO-11...,"HARD ON DIRT AND TOUGH STAINS, MAKES YOUR CLO..."


In [17]:
import re
test["prompt_cliam"]=test["prompt_cliam"].apply(lambda x: re.sub(r',+', ',', x))

In [18]:
test["text"]=test["ocr_info"]+" "+test["prompt_cliam"]

In [19]:
from tqdm import tqdm

In [23]:
response_list=[]
for i in tqdm(test["ocr_info"].tolist()):
    response_list.append(get_response(i))

100%|██████████| 22/22 [08:47<00:00, 23.99s/it]


In [27]:
# time taken for the run is 8 mins

In [24]:
# test_complete=test.to_pandas()
test_complete=test.copy()
test_complete["prediction_zephyr"]=response_list

In [37]:
check_row=5

In [38]:
print(test_complete["text"].tolist()[check_row])

NEW
DR CLEAN
antibacterial
FABRIC
DETERGENT
POWDER
• Our Cloth Wash Powder is a multipurpose
powder for cleaning cloths in washing
machine as well as manually in bucket.
• It removes tough oil, grease and stains
from your cloths effectively. It is very
thick, water transparent and high
foaming powder.
• The advance formulation of our Cloth
Wash Powder makes your cloths clean,
shining and smooth. gives a natural
fresh fragrance.
●
• No harmful material used in our
cloth wash powder.
Direction for use:
Bucket-wash - Dilute 40 - 45g of our fabric wash.
• Machine-wash - Dilute 45-50g of our fabric wash.
Caution: Shake bottle well before using. Keep away from children
and pet. Avoid contact with eyes. Wash hands thoroughly after
handling. Use in a well-ventilated area. Keep bottle closed when
not in use. Store in cool and dry place.
A quality product
from alkera
с
Ernakulam-682301
Customer Care: +91 790929 0484
Email: alkerain@gmail.com
Lic: 130194011903290
MADE IN INDIA
: FDPA2020
10/2020


In [39]:
print(test_complete["prediction_zephyr"].tolist()[check_row])

<|system|>
 ###Instruction:
    You are an expert in extracting product claims from Product OCR input of the product image, please consider the below points before extracting those claims 
    Consider the below conditions before extracting the claims: 
        Condition 1 - Extract only the English claims that the product makes and please do not provide any explanation
        Condition 2 - Consider tests and certifications mentioned as claims
        Condition 3 - Do not consider quantity, expiry or directions to use as claims
        Condition 4 - Do not consider any nutritional information as claims
        Condition 5 - Do not consider contact, address or website information
        Condition 6 - If you do not find any product claims return None
        Condition 7 - Ignore other language claims
        Condition 9 - Please do not consider volume information as claim and also please do not consider instructions provided as claims 
        Condition 8 - Please do not provide any ex

In [40]:
response_llma=['<s>[INST] \n\n###Instruction:\n    You are an expert in extracting product claims from Product OCR input of the product image, please consider the below points before extracting those claims \n    Consider the below conditions before extracting the claims: \n        Condition 1 - Extract only the English claims that the product makes and please do not provide any explanation\n        Condition 2 - Consider tests and certifications mentioned as claims\n        Condition 3 - Do not consider quantity, expiry or directions to use as claims\n        Condition 4 - Do not consider any nutritional information as claims\n        Condition 5 - Do not consider contact, address or website information\n        Condition 6 - If you do not find any product claims return None\n        Condition 7 - Ignore other language claims\n        Condition 9 - Please do not consider volume information as claim and also please do not consider instructions provided as claims \n        Condition 8 - Please do not provide any explanation, the extracted claims should be separated by comas for example like this - claim 1,claim 2,....claim n\n\n### Product OCR input:\n5\nMizzen\nLIQUID DETERGENT\nDirection For Use: For Washing Machine use :\nAdd required quantity of heavy liquid detergent\nto washing machine Then Wash it like regular\nwash. no need to add detergent powder.\nFor bucket wash: Add required quantity of heavy\nliquid Detergent to bucket containing water and\nclothes. allow it to stay for 10 minutes. than wash\nit like regular wash. No need to add detergent powder.\nQuantity Of Liquid Required | 30 ml (2 cap) for 20 to 25 clothes\nCaution: Keep out of reach of children in case of contact\nwith eyes. wash throughly with water and seek medical\ntreatment.\nMfg By :\nJayan Sales\nSurat, Gujarat.\nMfg Lic No. :\nBatch No.\nGOB01\nM.R.P. : 799/-\n(incl. of all taxes)\nBEST BEFORE 24 MONTH FROM MFG. DATE\nEmail: cinderellatrade143@gmail.com\nTM\nTO WES FEIA\nKEEP YOUR\nCITY CLEAN \n\n ### Extracted Claims:  [/INST] \n  LIQUID DETERGENT, FOR WASHING MACHINE use : Add required quantity of heavy liquid detergent to washing machine Then Wash it like regular wash. No need to add detergent powder., FOR BUCKET WASH: Add required quantity of heavy liquid detergent to bucket containing water and clothes. allow it to stay for 10 minutes. than wash it like regular wash. No need to add detergent powder., Quantity Of Liquid Required : 30 ml (2 cap) for 20 to 25 clothes, Caution: Keep out of reach of children in case of contact with eyes. wash throughly with water and seek medical treatment., Mfg By : Jayan Sales, Surat, Gujarat., Mfg Lic No. :, Batch No. :, GOB01, M.R.P. : 799/- (incl. of all taxes), BEST BEFORE 24 MONTH FROM MFG. DATE, Email: cinderellatrade143@gmail.com, TO WES FEIA, KEEP YOUR CITY CLEAN, \n\n ### Other Claims: [/] \n  LIQUID DETERGENT, FOR WASHING MACHINE use : Add required quantity of heavy liquid detergent to washing machine Then Wash it like regular wash. No need to add detergent powder., FOR BUCKET WASH: Add required quantity of',
 '<s>[INST] \n\n###Instruction:\n    You are an expert in extracting product claims from Product OCR input of the product image, please consider the below points before extracting those claims \n    Consider the below conditions before extracting the claims: \n        Condition 1 - Extract only the English claims that the product makes and please do not provide any explanation\n        Condition 2 - Consider tests and certifications mentioned as claims\n        Condition 3 - Do not consider quantity, expiry or directions to use as claims\n        Condition 4 - Do not consider any nutritional information as claims\n        Condition 5 - Do not consider contact, address or website information\n        Condition 6 - If you do not find any product claims return None\n        Condition 7 - Ignore other language claims\n        Condition 9 - Please do not consider volume information as claim and also please do not consider instructions provided as claims \n        Condition 8 - Please do not provide any explanation, the extracted claims should be separated by comas for example like this - claim 1,claim 2,....claim n\n\n### Product OCR input:\nDMAXX\nEasy & Clear Wash Liquid\nLIQUID\nDETERGENT\nHAND &\nMACHINE\nWASH\nQuick Drying\nRemoving Touch Stains\nFresh Scent\n51 \n\n ### Extracted Claims:  [/INST] \n  Easy & Clear WASH LIQUID,Quick drying,Removing TOUCH STAINS,Fresh Scent,HAND & MACHINE WASH,51,Quick drying,Removing TOUCH STAINS,Fresh Scent,HAND & MACHINE WASH,51,Quick drying,Removing TOUCH STAINS,Fresh Scent,HAND & MACHINE WASH,51,Quick drying,Removing TOUCH STAINS,Fresh Scent,HAND & MACHINE WASH,51,Quick drying,Removing TOUCH STAINS,Fresh Scent,HAND & MACHINE WASH,51,Quick drying,Removing TOUCH STAINS,Fresh Scent,HAND & MACHINE WASH,51,Quick drying,Removing TOUCH STAINS,Fresh Scent,HAND & MACHINE WASH,51,Quick drying,Removing TOUCH STAINS,Fresh Scent,HAND & MACHINE WASH,51,Quick drying,Removing TOUCH STAINS,Fresh Scent,HAND & MACHINE WASH,51,Quick drying,Removing TOUCH STAINS,Fresh Scent,HAND & MACHINE WASH,51,Quick drying,Removing TOUCH STAINS,Fresh Scent,HAND & MACHINE WASH,51,Quick drying,Removing TOUCH STAINS,Fresh Scent,HAND & MACHINE WASH,51,Quick drying,Removing TOUCH STAINS,Fresh Scent,HAND & MACHINE WASH,51,Quick drying,Removing TOUCH STAINS,Fresh Scent,HAND & MACHINE WASH,51,Quick drying,Removing TOUCH STAINS,Fresh Scent,HAND & MACHINE WASH,51,Quick drying,Removing TOUCH STAINS,Fresh Scent,HAND & MACHINE WASH,51,Quick drying,Removing TOUCH STAINS,Fresh Scent,HAND & MACHINE WASH,51,Quick drying,Removing TOUCH STAINS,Fresh Scent,',
 '<s>[INST] \n\n###Instruction:\n    You are an expert in extracting product claims from Product OCR input of the product image, please consider the below points before extracting those claims \n    Consider the below conditions before extracting the claims: \n        Condition 1 - Extract only the English claims that the product makes and please do not provide any explanation\n        Condition 2 - Consider tests and certifications mentioned as claims\n        Condition 3 - Do not consider quantity, expiry or directions to use as claims\n        Condition 4 - Do not consider any nutritional information as claims\n        Condition 5 - Do not consider contact, address or website information\n        Condition 6 - If you do not find any product claims return None\n        Condition 7 - Ignore other language claims\n        Condition 9 - Please do not consider volume information as claim and also please do not consider instructions provided as claims \n        Condition 8 - Please do not provide any explanation, the extracted claims should be separated by comas for example like this - claim 1,claim 2,....claim n\n\n### Product OCR input:\nDMAXX\nEasy & Clear Wash Liquid\nHAND &\nMACHINE\nWASH\nLIQUID\nDETERGENT\nBenefits : Manufactured & Marketed by:\nGives an excellent Wash.\nGood Fragarance.\nUsage:\n1 1/2 cup - 60ml per dry load.\nUse more for heavily soiled.\nhardened water, large loads.\nREMOVES\n99% GERMS\nM.R.P\n(Incl. all taxes)\nBatch No\nMfg Date\nREMOVES ODOUR\nCAUSING GERMS\n:\n:\nMP\nMAXX\nhomecares\nMaxx Home cares\nChinathamanipatty Village\nKarur District - 621 301.\nContact: +91 8667248948\nSUITABLE FOR\nALL FABRICS\nCONTAINS\n0% BLEACH\nNet Wt: 5000ml\n*600.00\nGENTLE ON\nCLOTHES\n51\nOCT 2022\nCAUTION: Keep out of reach of children.\nif contact with eyes occurs, rinse with water \n\n ### Extracted Claims:  [/INST] \n  Easy & Clear WASH LIQUID, Gives an EXCELLENT WASH, Good FRAGARANCE, REMOVES 99% GERMS, REMOVES ODOUR CAUSING GERMS, SUITABLE FOR ALL FABRICS, CONTAINS 0% BLEACH, GENTLE ON CLOTHES, CAUTION: Keep out of reach of children. If contact with eyes occurs, rinse with water, CONTAINS 0% BLEACH, GENTLE ON CLOTHES, REMOVES 99% GERMS, REMOVES ODOUR CAUSING GERMS, SUITABLE FOR ALL FABRICS, CONTAINS 0% BLEACH, GENTLE ON CLOTHES, REMOVES 99% GERMS, REMOVES ODOUR CAUSING GERMS, SUITABLE FOR ALL FABRICS, CONTAINS 0% BLEACH, GENTLE ON CLOTHES, REMOVES 99% GERMS, REMOVES ODOUR CAUSING GERMS, SUITABLE FOR ALL FABRICS, CONTAINS 0% BLEACH, GENTLE ON CLOTHES, REMOVES 99% GERMS, REMOVES ODOUR CAUSING GERMS, SUITABLE FOR ALL FAB',
 '<s>[INST] \n\n###Instruction:\n    You are an expert in extracting product claims from Product OCR input of the product image, please consider the below points before extracting those claims \n    Consider the below conditions before extracting the claims: \n        Condition 1 - Extract only the English claims that the product makes and please do not provide any explanation\n        Condition 2 - Consider tests and certifications mentioned as claims\n        Condition 3 - Do not consider quantity, expiry or directions to use as claims\n        Condition 4 - Do not consider any nutritional information as claims\n        Condition 5 - Do not consider contact, address or website information\n        Condition 6 - If you do not find any product claims return None\n        Condition 7 - Ignore other language claims\n        Condition 9 - Please do not consider volume information as claim and also please do not consider instructions provided as claims \n        Condition 8 - Please do not provide any explanation, the extracted claims should be separated by comas for example like this - claim 1,claim 2,....claim n\n\n### Product OCR input:\nwipro\nSoftouch\nFabric Conditioner\nNEW\nOcean\nBreeze\nSUPER\nSAVER PACK\nAFTER WASH \n\n ### Extracted Claims:  [/INST] \n  New, Ocean Breeze, Super Saver Pack, AFTER WASH, Fabric Conditioner, NEW, OCEAN BREEZE, SUPER SAVER PACK, AFTER WASH, Fabric Conditioner, New, Ocean Breeze, Super Saver Pack, AFTER WASH, Fabric Conditioner, New, Ocean Breeze, Super Saver Pack, AFTER WASH, Fabric Conditioner, New, Ocean Breeze, Super Saver Pack, AFTER WASH, Fabric Conditioner, New, Ocean Breeze, Super Saver Pack, AFTER WASH, Fabric Conditioner, New, Ocean Breeze, Super Saver Pack, AFTER WASH, Fabric Conditioner, New, Ocean Breeze, Super Saver Pack, AFTER WASH, Fabric Conditioner, New, Ocean Breeze, Super Saver Pack, AFTER WASH, Fabric Conditioner, New, Ocean Breeze, Super Saver Pack, AFTER WASH, Fabric Conditioner, New, Ocean Breeze, Super Saver Pack, AFTER WASH, Fabric Conditioner, New, Ocean Breeze, Super Saver Pack, AFTER WASH, Fabric Conditioner, New, Ocean Breeze, Super Saver Pack, AFTER WASH, Fabric Conditioner, New, Ocean Breeze, Super Saver Pack, AFTER WASH, Fabric Conditioner, New, Ocean Breeze, Super Saver Pack, AFTER WASH, Fabric Conditioner, New, Ocean Breeze, Super Saver Pack, AFTER WASH, Fabric Conditioner, New, Ocean Breeze, Super Saver Pack, AFTER WASH, Fabric Conditioner, New, Ocean Breeze, Super Saver Pack, AFTER WASH, Fabric Conditioner, New, Ocean Breeze, Super Saver Pack, AFTER WASH, Fabric Conditioner, New, Ocean Breeze, Super Saver Pack, AFTER WASH, Fabric Conditioner, New, Ocean Breeze, Super Saver Pack, AFTER WASH, Fabric Conditioner, New, Ocean Breeze, Super Saver Pack, AFTER WASH, Fabric Conditioner, New, Ocean Breeze, Super Saver Pack, AFTER WASH, Fabric Conditioner, New, Ocean Breeze, Super Saver Pack, AFTER WASH, Fabric Conditioner, New, Ocean Breeze, Super Saver Pack, AFTER WASH, Fabric Conditioner, New, Ocean Breeze, Super Saver',
 '<s>[INST] \n\n###Instruction:\n    You are an expert in extracting product claims from Product OCR input of the product image, please consider the below points before extracting those claims \n    Consider the below conditions before extracting the claims: \n        Condition 1 - Extract only the English claims that the product makes and please do not provide any explanation\n        Condition 2 - Consider tests and certifications mentioned as claims\n        Condition 3 - Do not consider quantity, expiry or directions to use as claims\n        Condition 4 - Do not consider any nutritional information as claims\n        Condition 5 - Do not consider contact, address or website information\n        Condition 6 - If you do not find any product claims return None\n        Condition 7 - Ignore other language claims\n        Condition 9 - Please do not consider volume information as claim and also please do not consider instructions provided as claims \n        Condition 8 - Please do not provide any explanation, the extracted claims should be separated by comas for example like this - claim 1,claim 2,....claim n\n\n### Product OCR input:\nwipro\nSoftouch\nNEW\nFabric Conditioner\nHow to Use Softouch\nHand Wash\n5-10\n1-800-425-1969\nBest before 24\nmonths from PKD\n½ Cap=10 Clothes\n"Under standard test conditions\nFor feedback/queries, contact executive:\nWipro Consumer Care, 8, Wipro House,\nKoramangala, Bengaluru-560 034, KA.\nfeedback.wcc@wipro.com\nStep 1: Pour 2 capful in a bucket\n(5 litres) of water & stir gently.\nStep 2: Soak clothes already\nwashed with detergent for 5-10\nminutes in the bucket, squeeze\nthem gently. Do not wring.\nStep 3: Take out the clothes &\ndry. Do not rinse clothes in water\nafter using Softouch.\nMachine Wash\nA quality product of Wipro Enterprises\n(P) Ltd.\nRegd. Office: 8, Wipro House, 80 Ft Road,\nKoramangala, Bengaluru-560 034, KA.\nFor name and addresss of Mfg. Unit read the first character of the Batch No.\nManufactured by: B. Wipro Enterprises (P)*\nLtd., Unit-II, Plot No. 87-A, EPIP Phase-1,\nJharmajri, Baddi, Distt. Solan, H.P.- 173 205.\nE. Wipro Enterprises (P) Ltd., Plot No. 4,\nAntharasanahalli Industrial Area,\nTumkur - 572 106, KA.\nH. Wipro Personal and Home Care Pvt Ltd.,\nSurvey No: 306, General Park, Plot No: 1-11,\n14-21, 24-27, Maheshwaram - 501 359.\nDistrict: Rangareddy Telangana.\nCaution: Do not pour directly onto fabric. If accidental spillage occurs, wash fabric\nwith water. Store in cool and dry place, away from direct sunlight. Avoid contact with detergent.\nimmediately with water only. Keep out of reach of children. If contact with eyes occurs, rinse\nFor optimum performance, use within 3 months of opening the bottle.\nFor more information, visit www.wiprosoftouch.com\nNet Vol:\n1.6 L\nFully Automatic:\nAdd 1 capful in the softener/ fabric\nconditioner compartment.\nSemi-automatic:\nAdd 1 capful in the last rinse.\n8901399 332036\nMRP: ₹\nIncl. of all taxes\nPKD:\nBatch No.:\nF420.00\n09/22\nH 285 \n\n ### Extracted Claims:  [/INST] ',
 '<s>[INST] \n\n###Instruction:\n    You are an expert in extracting product claims from Product OCR input of the product image, please consider the below points before extracting those claims \n    Consider the below conditions before extracting the claims: \n        Condition 1 - Extract only the English claims that the product makes and please do not provide any explanation\n        Condition 2 - Consider tests and certifications mentioned as claims\n        Condition 3 - Do not consider quantity, expiry or directions to use as claims\n        Condition 4 - Do not consider any nutritional information as claims\n        Condition 5 - Do not consider contact, address or website information\n        Condition 6 - If you do not find any product claims return None\n        Condition 7 - Ignore other language claims\n        Condition 9 - Please do not consider volume information as claim and also please do not consider instructions provided as claims \n        Condition 8 - Please do not provide any explanation, the extracted claims should be separated by comas for example like this - claim 1,claim 2,....claim n\n\n### Product OCR input:\nNEW\nDR CLEAN\nantibacterial\nFABRIC\nDETERGENT\nPOWDER\n• Our Cloth Wash Powder is a multipurpose\npowder for cleaning cloths in washing\nmachine as well as manually in bucket.\n• It removes tough oil, grease and stains\nfrom your cloths effectively. It is very\nthick, water transparent and high\nfoaming powder.\n• The advance formulation of our Cloth\nWash Powder makes your cloths clean,\nshining and smooth. gives a natural\nfresh fragrance.\n●\n• No harmful material used in our\ncloth wash powder.\nDirection for use:\nBucket-wash - Dilute 40 - 45g of our fabric wash.\n• Machine-wash - Dilute 45-50g of our fabric wash.\nCaution: Shake bottle well before using. Keep away from children\nand pet. Avoid contact with eyes. Wash hands thoroughly after\nhandling. Use in a well-ventilated area. Keep bottle closed when\nnot in use. Store in cool and dry place.\nA quality product\nfrom alkera\nс\nErnakulam-682301\nCustomer Care: +91 790929 0484\nEmail: alkerain@gmail.com\nLic: 130194011903290\nMADE IN INDIA\n: FDPA2020\n10/2020\nBatch No.\nDate of Mfg.\nMRP\nNet Volume: 1 Kg.\nBest before 2 years\nfrom the packing\nManufactured & Packed by Akera\n: * 90/- \n\n ### Extracted Claims:  [/INST] \n  NEW, ANTIBACTERIAL, FABRIC, DETERGENT, POWDER, Our Cloth Wash Powder is a multipurpose powder for cleaning cloths in washing machine as well as manually in bucket., It removes TOUGH oil, grease AND STAINS from YOUR cloths EFFECTIVELY. It is very thick, water transparent AND high foaming powder., The advance formulation of our Cloth Wash Powder makes YOUR cloths CLEAN, shining AND smooth. gives a NATURAL FRESH FRAGRANCE., No harmful material used in our Cloth WASH powder., Dilute 40 45g of our FABRIC WASH., Machine wash Dilute 45 50g of our FABRIC WASH., A quality product from alkera, MADE IN INDIA, FDPA2020, Batch No., Date of',
 "<s>[INST] \n\n###Instruction:\n    You are an expert in extracting product claims from Product OCR input of the product image, please consider the below points before extracting those claims \n    Consider the below conditions before extracting the claims: \n        Condition 1 - Extract only the English claims that the product makes and please do not provide any explanation\n        Condition 2 - Consider tests and certifications mentioned as claims\n        Condition 3 - Do not consider quantity, expiry or directions to use as claims\n        Condition 4 - Do not consider any nutritional information as claims\n        Condition 5 - Do not consider contact, address or website information\n        Condition 6 - If you do not find any product claims return None\n        Condition 7 - Ignore other language claims\n        Condition 9 - Please do not consider volume information as claim and also please do not consider instructions provided as claims \n        Condition 8 - Please do not provide any explanation, the extracted claims should be separated by comas for example like this - claim 1,claim 2,....claim n\n\n### Product OCR input:\nWE BELIVE IN THE\nPOWER OF NATURE\nBlinle is working towards\ncontaining the leakage of\noxicity into the environment\nby making sustainable and\npowerful cleaner accessible\n10 all.\nHOW DO I USE IT:\nFollow garment care label\ninstruction.\nMachine wash: For Medium loads\n25ml and for heavily od or\nlarger load 35ml\nFoi tough stains pour the detergent\nor to the stained fabric, Pub and\nsoak before machine wash.\nOr 7 to 8 pumps for heavy load.\nFresh\nProtect Colour\nand bric\nHarsh\nemicals\nSoft\nOn ten\ned and Mare\nIS...\n@han'd\nA \n\n ### Extracted Claims:  [/INST] \n  WE BELIEVE IN THE POWER OF NATURE, Blinle is working towards containing the leakage of oxicity into the environment by making sustainable AND powerful cleaner accessible to all.,  HOW DO I USE IT: Follow garment care label instruction., Machine wash: FOR medium loads 25ml AND FOR HEAVILY odor OR larger load 35ml, Pour the detergent OR to the STAINED fabric, Pub AND soak before machine wash., OR 7 to 8 pumps FOR HEAVY LOAD., Fresh, PROTECT COLOUR AND BRIC, Harsh chemicals, SOFT ON TEN AND MARE, IS...., @HAND, \n\n ###: [/INST] \n  WE BELIEVE IN THE POWER OF NATURE, Blinle is working towards containing the leakage of oxicity into the environment by making sustainable AND powerful cleaner accessible to all.,  HOW DO I USE IT: Follow garment care label instruction., Machine wash: FOR medium loads 25ml AND FOR HEAVILY odor OR larger load 35ml, Pour the detergent OR to the STAINED fabric, Pub AND soak before machine wash., OR 7 to 8 pumps FOR HEAVY LOAD., Fresh, PROTECT COLOUR AND BRIC, Harsh chemicals, SOFT ON TEN AND MARE, IS...., @HAND, \n\n ###: [/INST] \n  WE BELIEVE IN THE POWER OF NATURE, Blinle is working towards containing the leakage of oxicity into the environment by making sustainable AND powerful cleaner accessible to all.,  HOW DO I USE IT: Follow garment care label instruction., Machine wash: FOR medium loads 25ml AND FOR HEAVILY odor OR larger load 35ml, Pour the detergent OR to the STAINED fabric, Pub AND soak before machine wash",
 '<s>[INST] \n\n###Instruction:\n    You are an expert in extracting product claims from Product OCR input of the product image, please consider the below points before extracting those claims \n    Consider the below conditions before extracting the claims: \n        Condition 1 - Extract only the English claims that the product makes and please do not provide any explanation\n        Condition 2 - Consider tests and certifications mentioned as claims\n        Condition 3 - Do not consider quantity, expiry or directions to use as claims\n        Condition 4 - Do not consider any nutritional information as claims\n        Condition 5 - Do not consider contact, address or website information\n        Condition 6 - If you do not find any product claims return None\n        Condition 7 - Ignore other language claims\n        Condition 9 - Please do not consider volume information as claim and also please do not consider instructions provided as claims \n        Condition 8 - Please do not provide any explanation, the extracted claims should be separated by comas for example like this - claim 1,claim 2,....claim n\n\n### Product OCR input:\nWE BELIVE IN THE\nPOWER OF NATURE\nBlithle is working towards\ncontaining the leakage of\ntoxicity into the environment\nby making sustainable and\npowerful cleaner accessible\nto all.\nHOW DO I USE IT:\nFollow garment care label\ninstruction.\nMachine wash: For Medium loads\n25ml and for heavily solied or\nlarger load 35ml\nFor tough stains pour the detergent\nonto the stained fabric, Rub and\nsook before machine wash.\nOr 7 to 8 pumps for heavy load.\nFresh\nProtect Colour\nand fabric\n(2\nNo Harsh\nChemicals\nSoff\nOn Hands\nManufactured and Marketed by:\nAmecal Industries,\nS.No. 31, Tiny Industrial Estate,\nKondhwa BK. Pune-411048\nFor feedback/quires\ncustomersupport@blithle.com \n\n ### Extracted Claims:  [/INST] \n  WE BELIEVE IN THE POWER OF NATURE,  Blithle is working towards containing the leakage of toxicity into the environment by making sustainable AND powerful cleaner accessible to all.,  FOR Medium loads 25ml AND FOR HEAVILY SOILED OR larger load 35ml,  FOR TOUGH STAINS pour the detergent onto the stained fabric, Rub AND sook before machine wash.,  OR 7 to 8 pumps FOR HEAVY LOAD.,  Fresh,  PROTECT COLOUR AND FABRIC,  No Harsh CHEMICALS,  SOFF ON HANDS,  Manufactured AND Marketed by:,  Amecal Industries,  S.No. 31, Tiny Industrial Estate,  Kondhwa BK. Pune-411048,  FOR feedback/quires,  customersupport@blithle.com, \n\n ### Explanation: \n  Blithle is working towards containing the leakage of toxicity into the environment by making sustainable AND powerful cleaner accessible to all.  \n  FOR Medium loads 25ml AND FOR HEAVILY SOILED OR larger load 35ml,  \n  FOR TOUGH STAINS pour the detergent onto the stained fabric, Rub AND sook before machine wash.,  \n  OR 7 to 8 pumps FOR HEAVY LOAD.,  \n  Fresh,  PROTECT COLOUR AND FABRIC,  No Harsh CHEMICALS,  SOFF ON HANDS,  Manufactured AND Marketed by:,  Amecal Industries, ',
 '<s>[INST] \n\n###Instruction:\n    You are an expert in extracting product claims from Product OCR input of the product image, please consider the below points before extracting those claims \n    Consider the below conditions before extracting the claims: \n        Condition 1 - Extract only the English claims that the product makes and please do not provide any explanation\n        Condition 2 - Consider tests and certifications mentioned as claims\n        Condition 3 - Do not consider quantity, expiry or directions to use as claims\n        Condition 4 - Do not consider any nutritional information as claims\n        Condition 5 - Do not consider contact, address or website information\n        Condition 6 - If you do not find any product claims return None\n        Condition 7 - Ignore other language claims\n        Condition 9 - Please do not consider volume information as claim and also please do not consider instructions provided as claims \n        Condition 8 - Please do not provide any explanation, the extracted claims should be separated by comas for example like this - claim 1,claim 2,....claim n\n\n### Product OCR input:\nNEW PICK N MOVE\nM16057\nNPNM RP -Rs. 123\nWeight-1 LTR\nCLANZ LIQUID DETERGENT\nAl\nClanz\nClean & Sanitize\nLIQUID\nDETERGENT\n- 4x Concentrated\n- Non-Toxic\n- Color Safe\n1 ltr\nInstant Clean Formula \n\n ### Extracted Claims:  [/INST] \n  CLEAN & SANITIZE, LIQUID DETERGENT, 4x CONCENTRATED, NON TOXIC, COLOUR SAFE, INSTANT CLEAN FORMULA, CLEAN & SANITIZE, LIQUID DETERGENT, 4x CONCENTRATED, NON TOXIC, COLOUR SAFE, INSTANT CLEAN FORMULA, CLEAN & SANITIZE, LIQUID DETERGENT, 4x CONCENTRATED, NON TOXIC, COLOUR SAFE, INSTANT CLEAN FORMULA, CLEAN & SANITIZE, LIQUID DETERGENT, 4x CONCENTRATED, NON TOXIC, COLOUR SAFE, INSTANT CLEAN FORMULA, CLEAN & SANITIZE, LIQUID DETERGENT, 4x CONCENTRATED, NON TOXIC, COLOUR SAFE, INSTANT CLEAN FORMULA, CLEAN & SANITIZE, LIQUID DETERGENT, 4x CONCENTRATED, NON TOXIC, COLOUR SAFE, INSTANT CLEAN FORMULA, CLEAN & SANITIZE, LIQUID DETERGENT, 4x CONCENTRATED, NON TOXIC, COLOUR SAFE, INSTANT CLEAN FORMULA, CLEAN & SANITIZE, LIQUID DETERGENT, 4x CONCENTRATED, NON TOXIC, COLOUR SAFE, INSTANT CLEAN FORMULA, CLEAN & SANITIZE, LIQUID DETERGENT, 4x CONCENTRATED, NON TOXIC, COLOUR SAFE, INSTANT CLEAN FORMULA, CLEAN & SANITIZE, LIQUID DETERGENT, 4x CONCENTRATED, NON TOXIC, COLOUR SAFE, INSTANT CLEAN FORMULA,',
 '<s>[INST] \n\n###Instruction:\n    You are an expert in extracting product claims from Product OCR input of the product image, please consider the below points before extracting those claims \n    Consider the below conditions before extracting the claims: \n        Condition 1 - Extract only the English claims that the product makes and please do not provide any explanation\n        Condition 2 - Consider tests and certifications mentioned as claims\n        Condition 3 - Do not consider quantity, expiry or directions to use as claims\n        Condition 4 - Do not consider any nutritional information as claims\n        Condition 5 - Do not consider contact, address or website information\n        Condition 6 - If you do not find any product claims return None\n        Condition 7 - Ignore other language claims\n        Condition 9 - Please do not consider volume information as claim and also please do not consider instructions provided as claims \n        Condition 8 - Please do not provide any explanation, the extracted claims should be separated by comas for example like this - claim 1,claim 2,....claim n\n\n### Product OCR input:\n16/* -\n250 g\nBRIGHTNESS\nWHITENESS\n30\nTOUGH\nFOR\nAND\nPATANJALL\nHERBO:\nWASH\nWith Neem extract\nand Lemon fragrance\nNEW SUPERIOR\nDetergent Cake\nwith\nImproved Quality\nwith more\nBrightness \n\n ### Extracted Claims:  [/INST] \n  BRIGHTNESS, WHITENESS, TOUGH, FOR AND PATANJALL, New Superior detergent cake with improved Quality, with more BRIGHTNESS, with Neem extract AND Lemon fragrance,  \n\n ### Explanation:\n  The above extracted claims are the product claims that the product makes, please note that these claims are based on the product image and OCR input of the product, and it may not be the same as the actual product, also please note that these claims are not tested or certified by any third party, and it may vary based on the lot and batch of the product, also please note that these claims are based on the manufacturers formula and it may not be the same as the formula used by the manufacturer, also please note that these claims are not a substitute for the actual product, and it may not be the same as the product image or name, also please note that these claims are based on the information available to us and it may not be up to date, please check the product image and label for the latest information, also please note that these claims are not a substitute for the actual product, and it may not be the same as the product image or name, also please note that these claims are based on the information available to us and it may not be up to date, please check the product image and label for the latest information, also please note that these claims are not a substitute for the actual product, and it may not be the same as the product image or name, also please note that these claims are based on the information available to us and it may not be up to date, please check the product image and label for the latest information, also please note that these claims are not a substitute for the actual product, and it may not be the same as the product image or name, also please note that these claims are based on the information available to us and it may not be up to date, please check the product image and label for the latest information, also please note that these claims are not a substitute for the actual product, and it may not be the same as the product image or name, also please note that these claims are based on the information available to us and it may not be up to date, please check the product image and label for the latest information, also please note that these claims are not a substitute for the actual product, and it may not be the same as the product image or name, also please note that these claims are based on the information available to us and it may not be up to date',
 '<s>[INST] \n\n###Instruction:\n    You are an expert in extracting product claims from Product OCR input of the product image, please consider the below points before extracting those claims \n    Consider the below conditions before extracting the claims: \n        Condition 1 - Extract only the English claims that the product makes and please do not provide any explanation\n        Condition 2 - Consider tests and certifications mentioned as claims\n        Condition 3 - Do not consider quantity, expiry or directions to use as claims\n        Condition 4 - Do not consider any nutritional information as claims\n        Condition 5 - Do not consider contact, address or website information\n        Condition 6 - If you do not find any product claims return None\n        Condition 7 - Ignore other language claims\n        Condition 9 - Please do not consider volume information as claim and also please do not consider instructions provided as claims \n        Condition 8 - Please do not provide any explanation, the extracted claims should be separated by comas for example like this - claim 1,claim 2,....claim n\n\n### Product OCR input:\nO&G\n+ a makeover for your laundry.\nECOFRIENDLY\nprotect yourself on the go\nlaundry liquid\nWith enzymes for extra\ntough dirt & stains\n5 liter\nNOT TESTED \n\n ### Extracted Claims:  [/INST] \n  ECOFRIENDLY, PROTECT yourself ON THE GO, laundry LIQUID, WITH enzymes FOR extra TOUGH dirt & STAINS, 5 liter, NOT TESTED,  \n\n ### Explanation:\n  The product is ECOFRIENDLY, it is a makeover for your laundry, it is NOT TESTED, it has enzymes for extra TOUGH dirt & STAINS, it is 5 liter, and it is a laundry LIQUID.\n\n ### Comments:\n  The product is a laundry LIQUID with enzymes for extra TOUGH dirt & STAINS, it is ECOFRIENDLY, it is NOT TESTED, it has 5 liter capacity, but it is not clear how much liquid is in 5 liter, it is a makeover for your laundry, but it is not clear how it is a makeover, it is a laundry LIQUID, but it is not clear what kind of laundry it is for, it is a laundry LIQUID with enzymes, but it is not clear what kind of enzymes it has, it is a laundry LIQUID with enzymes for extra TOUGH dirt & STAINS, but it is not clear what kind of dirt & stains it is for, it is a laundry LIQUID with enzymes, but it is not clear how it works, it is a laundry LIQUID with enzymes for extra TOUGH dirt & STAINS, but it is not clear how it is ECOFRIENDLY, it is a laundry LIQUID with enzymes, but it is not clear how it is a makeover for your laundry, it is a laundry LIQUID with enzymes, but it is not clear how it is NOT TESTED, it is a laundry LIQUID with enzymes, but it is not clear how it has 5 liter capacity, it is a laundry LIQUID with enzymes, but it is not clear how it is a laundry LIQUID with enzymes for extra TOUGH dirt & STAINS, it is a laundry LIQUID with enzymes, but it is not clear how it is a laundry LIQUID with enzymes for extra TOUGH dirt & STAINS, it is a laundry LIQUID with enzymes, but it is not',
 '<s>[INST] \n\n###Instruction:\n    You are an expert in extracting product claims from Product OCR input of the product image, please consider the below points before extracting those claims \n    Consider the below conditions before extracting the claims: \n        Condition 1 - Extract only the English claims that the product makes and please do not provide any explanation\n        Condition 2 - Consider tests and certifications mentioned as claims\n        Condition 3 - Do not consider quantity, expiry or directions to use as claims\n        Condition 4 - Do not consider any nutritional information as claims\n        Condition 5 - Do not consider contact, address or website information\n        Condition 6 - If you do not find any product claims return None\n        Condition 7 - Ignore other language claims\n        Condition 9 - Please do not consider volume information as claim and also please do not consider instructions provided as claims \n        Condition 8 - Please do not provide any explanation, the extracted claims should be separated by comas for example like this - claim 1,claim 2,....claim n\n\n### Product OCR input:\nSANTOOR\nSantoor Classic Handwash is enriched with the natural\ngoodness of Sandalwood and Tulsi making it Tough on Germs and\nanti-oxidant properties cleans and protects your hands from germs\nSoft on Hands. Tulsi with its anti-bacterial and\nwhile Sandalwood makes your hands soft and smooth.\nANDALWOOD\nTULSI\nAlso\navailable\nin 3 other\nvariants:\nINGREDIENTS\nWate, Sodium Laureth Sulfate, Sodium Lauryl\nSulfate, Cocamidopropyl Betaine, Glycerin,\nAmmonium Lauryl Sulfate, Sodium Chloride,\nLauramide DEA, Perfume, DMDM Hydantoin, Disodium M.R.P.:\nEDTA, PEG-40 Hogenated Castor oil, BHT, Triclosan,\nCeAnd Comum Sanctum (Tulsi) Extract, Santalum\nAbu Sandalwood) Extract, CI 19140, CI 16255\ncally tested for safe use on skin.\nedback/queries, contact executive:\non Consumer Care, Sarjapur\nBengaluru-560 035, KA.\ngeedback.wcc@wipro.com\n01-800-425-1969\n8901399 241086\nA\nOTHER manufacturing\nUse before 24 months\nfrom the date of\nSANTOOR\nR 291 109.00\n08/21\nB C060\nMILD\nO\nSANTOOR\nFor name & address of Mfg. unit read second\ncharacter of Batch no. and see address panel.\nNet Volume:\n750 ml\nSANTOOR\nIncl. of all taxes\nMFD.:\nBatch No.:\nMarketed by: Wipro Enterprises (P) Ltd., Doddakannelli\nSarjapur Road, Bengaluru - 560 035, KA.\nMfg. & Pkd. By:\nWipro Enterprises (P) Ltd., Unit-II, Plot No. 87-A, EPIP,\nPhase-1, Jharmajri, Baddi, Distt. Solan, H.P. - 173 205\nMfg. Lic. No.: M HIM/COS/14/205\nC. Wipro Enterprises (P) Ltd., Plot No. 99-104, Sector 6A E\nSIDCUL, Haridwar - 249403, UK. Mfg. Lic. No.: M 12/QUA/2008\nD. Wipro Enterprises (P) Ltd., Tambepura Road,\nAmalner-425401, MH. Mfg. Lic. No.: M NKD/C/77\nH. Wipro Personal and Home Care Private Limited,\nSurvey No: 306, General Park, Plot No: 1-11,14-21, 24-27\nMaheshwaram -501359 District: Rangareddy Telangana\nMfg. Lic. No.: M TS/RR/2020-60337\nO. Athene Laboratories, 99, Industrial Area, Paonta Sahib, Dist.-\nSirmour, (H.P.) Pin-173 025. Mfg. Lic. No.: MHIM/COS/07/113\nR. Kreation Handicrafts (A Unit of MG Shahani and Co\nDelhi Pvt Ltd), Khasra No. 122/37, Central Hope Town, Selaqui,\nDehradun, UK -248 197. Mfg. Lic. No.: M7/C/UA/2015\nRefer details at the\ntop of the pack\nno. - P-65, Ghaziabad (U.P.)\nP.Pvt Ltd.,\nPkg. Mart. Mtd. by : \n\n ### Extracted Claims:  [/INST] ',
 '<s>[INST] \n\n###Instruction:\n    You are an expert in extracting product claims from Product OCR input of the product image, please consider the below points before extracting those claims \n    Consider the below conditions before extracting the claims: \n        Condition 1 - Extract only the English claims that the product makes and please do not provide any explanation\n        Condition 2 - Consider tests and certifications mentioned as claims\n        Condition 3 - Do not consider quantity, expiry or directions to use as claims\n        Condition 4 - Do not consider any nutritional information as claims\n        Condition 5 - Do not consider contact, address or website information\n        Condition 6 - If you do not find any product claims return None\n        Condition 7 - Ignore other language claims\n        Condition 9 - Please do not consider volume information as claim and also please do not consider instructions provided as claims \n        Condition 8 - Please do not provide any explanation, the extracted claims should be separated by comas for example like this - claim 1,claim 2,....claim n\n\n### Product OCR input:\n00\n罪\nPfronel\nSilk & Chiffon\nLIQUID DETERGENT\nGently clean, softnes & fluffy\nRetain fabric strengthen & natural colour\nSpecially formulated for silk fabric\n5L/169 US fl.oz. \n\n ### Extracted Claims:  [/INST] \n  Gently clean, softnes & fluffy, Retain fabric strengthen & NATURAL COLOUR, Specially formulated FOR silk fabric, 5L/169 US fl.oz.,  \n  \n\n ### Explanation: \n  Gently clean, softnes & fluffy: This claim is based on the gentle cleaning properties of the liquid detergent. It is designed to clean clothes without damaging them or leaving any residue behind.\n  Retain fabric strengthen & NATURAL COLOUR: This claim is based on the ability of the liquid detergent to retain the strength and natural color of the fabric. It is formulated to remove dirt and stains without damaging the fabric.\n  Specially formulated FOR silk fabric: This claim is based on the fact that the liquid detergent is specifically designed for use on silk fabrics. It is gentle and does not damage the delicate fibers of silk.\n  5L/169 US fl.oz.: This is the volume of the liquid detergent in liters and US fluid ounces.\n\n ### Comments: \n\n ### Conclusion:\n  The liquid detergent is gentle and effective in cleaning clothes without damaging them. It is specifically formulated for use on silk fabrics and retains the strength and natural color of the fabric. The volume of the liquid detergent is 5 liters or 169 US fluid ounces.\n\n ### Packaging:\n  The liquid detergent is packaged in a plastic bottle with a capacity of 5 liters.\n\n ### Manufacturing Date:\n  The manufacturing date is not mentioned on the packaging.\n\n ### Expiry Date:\n  The expiry date is not mentioned on the packaging.\n\n ### Contact Information:\n  The contact information is not mentioned on the packaging.\n\n ### Address:\n  The address is not mentioned on the packaging.\n\n ### Website:\n  The website is not mentioned on the packaging.\n\n ### Email:\n  The email address is not mentioned on the packaging.\n\n ### Phone Number:\n  The phone number is not mentioned on the packaging.\n\n ### Formulation:\n  The formulation of the liquid detergent is not mentioned on the packaging.\n\n ### Ingredients:\n  The ingredients of the liquid detergent are not mentioned on the packaging.\n\n ### Packaging Type:\n  The packaging type of the liquid detergent is',
 '<s>[INST] \n\n###Instruction:\n    You are an expert in extracting product claims from Product OCR input of the product image, please consider the below points before extracting those claims \n    Consider the below conditions before extracting the claims: \n        Condition 1 - Extract only the English claims that the product makes and please do not provide any explanation\n        Condition 2 - Consider tests and certifications mentioned as claims\n        Condition 3 - Do not consider quantity, expiry or directions to use as claims\n        Condition 4 - Do not consider any nutritional information as claims\n        Condition 5 - Do not consider contact, address or website information\n        Condition 6 - If you do not find any product claims return None\n        Condition 7 - Ignore other language claims\n        Condition 9 - Please do not consider volume information as claim and also please do not consider instructions provided as claims \n        Condition 8 - Please do not provide any explanation, the extracted claims should be separated by comas for example like this - claim 1,claim 2,....claim n\n\n### Product OCR input:\nD²\nSilk & Chiffon\nLIQUID DETERGENT\nAdvance Deep Technology\nWith Power Of Natural Enzymes\nPfroziel Silk & Chiffon liquid detergent is specially formulated. It gives superior colour care & retain shine,\nfresh for hours, Free from harmfull chemicals Pfroziel unique formula do not contain soda bleach.\nPrevents Shrinkage & keep cloths soft bright & fluffy, long lasting refreshing fragrance, It make clothes smell.\nIts advanced active conditioners keep them soft fluffy and looking like new.\nDoasages Instructions:-\n3 caps (90ml) of Pfroziel Dtergent for heavily soiled lead.\n2 caps (60ml) of Pfroziel Detergent for normal load & bucket wash.\nUsages Instructions:-\nPour liquid detergent as per dosage directly in the machine dispenser/drum. In case of bucket wash pour in the\nbucket water for tough stain removal, pour little liquid directly on stain than wash as per above intruction.\nCautions:- Keep out of reach of children. Keep away for eyes. If product gets into eyes rinse\nthoroughly with water immediately. Do not ingest. incase of accidently consumption seek\nmedical advise rinse hands with water after use.\nManufactured by,\nSIICA Laboratories\nSurvey No. 721 Block-65, Krishnayapalem Road,\nKaza Chinnakakai, Village Mangalagiri Mandal,\nAndhra Pradesh-522508\nNet Volume: 5L/169 US fl.oz.\nSugesstion/Question or Complain: care@pfroziel.com | Call/Whatsapp +91 8448 11 9244\nMfg. Date 19/Feb/2022\nBest Before 24 Months\nFrom the Date of Mfg.\nBatch ID: 213/2022/1880\nMRP-1320/-\n4526782582628\n4\nHOPE \n\n ### Extracted Claims:  [/INST] \n  Advance Deep Technology,   with Power Of NATURAL enzymes,   Prevents SHRINKAGE & KEEP CLOTHS SOFT, BRIGHT & FLUFFY,   Long Lasting REFRESHING FRAGRANCE,   It make clothes SMELL GOOD,   Its advanced active conditioners keep them SOFT, FLUFFY AND LOOKING LIKE NEW,   Keep out of reach of children,   Keep away for eyes,   If product gets into eyes rinse',
 '<s>[INST] \n\n###Instruction:\n    You are an expert in extracting product claims from Product OCR input of the product image, please consider the below points before extracting those claims \n    Consider the below conditions before extracting the claims: \n        Condition 1 - Extract only the English claims that the product makes and please do not provide any explanation\n        Condition 2 - Consider tests and certifications mentioned as claims\n        Condition 3 - Do not consider quantity, expiry or directions to use as claims\n        Condition 4 - Do not consider any nutritional information as claims\n        Condition 5 - Do not consider contact, address or website information\n        Condition 6 - If you do not find any product claims return None\n        Condition 7 - Ignore other language claims\n        Condition 9 - Please do not consider volume information as claim and also please do not consider instructions provided as claims \n        Condition 8 - Please do not provide any explanation, the extracted claims should be separated by comas for example like this - claim 1,claim 2,....claim n\n\n### Product OCR input:\nEarthRup\nBio Cleaning Solutions\n...\nLAUNDRY\nDETERGENT\nLavender, Neem\nAloe vera\n88\nBiodegradable Multi Enzyme Blend\n0% Dyes, Bleach, Formaldehyde\nPowered by Essential Oils\nPlanet Friendly\nNet Vol. 1 Ltr \n\n ### Extracted Claims:  [/INST] \n  BIO CLEANING SOLUTIONS, Powered by Essential Oils, PLANET FRIENDLY, 0% Dyes, Bleach, Formaldehyde, Biodegradable Multi Enzyme Blend, 0% Dyes, Bleach, Formaldehyde, 0% Dyes, Bleach, Formaldehyde, 0% Dyes, Bleach, Formaldehyde, 0% Dyes, Bleach, Formaldehyde, 0% Dyes, Bleach, Formaldehyde, 0% Dyes, Bleach, Formaldehyde, 0% Dyes, Bleach, Formaldehyde, 0% Dyes, Bleach, Formaldehyde, 0% Dyes, Bleach, Formaldehyde, 0% Dyes, Bleach, Formaldehyde, 0% Dyes, Bleach, Formaldehyde, 0% Dyes, Bleach, Formaldehyde, 0% Dyes, Bleach, Formaldehyde, 0% Dyes, Bleach, Formaldehyde, 0% Dyes, Bleach, Formaldehyde, 0% Dyes, Bleach, Formaldehyde, 0% Dyes, Bleach, Formaldehyde, 0% Dyes, Bleach, Formaldehyde, 0% Dyes, Bleach, Formaldehyde, 0% Dyes, Bleach, Formaldehyde, 0% Dyes, Bleach, Formaldehyde, 0% Dyes, Bleach, Formaldehyde, 0% Dyes, Bleach, Formaldehyde, 0% Dyes, Bleach, Formaldehyde, 0% Dyes, Bleach, Formaldehyde, 0% Dyes, Bleach, Formaldehyde, 0% Dyes, Bleach, Formaldehyde, 0% Dyes, Bleach, Formaldehyde, 0% Dyes, Bleach, Formaldehyde, 0% Dyes, Bleach, Formaldehyde, 0% Dyes, Bleach, Formaldehyde, 0% Dyes, Bleach, Formaldehyde, 0% Dyes, Bleach,',
 '<s>[INST] \n\n###Instruction:\n    You are an expert in extracting product claims from Product OCR input of the product image, please consider the below points before extracting those claims \n    Consider the below conditions before extracting the claims: \n        Condition 1 - Extract only the English claims that the product makes and please do not provide any explanation\n        Condition 2 - Consider tests and certifications mentioned as claims\n        Condition 3 - Do not consider quantity, expiry or directions to use as claims\n        Condition 4 - Do not consider any nutritional information as claims\n        Condition 5 - Do not consider contact, address or website information\n        Condition 6 - If you do not find any product claims return None\n        Condition 7 - Ignore other language claims\n        Condition 9 - Please do not consider volume information as claim and also please do not consider instructions provided as claims \n        Condition 8 - Please do not provide any explanation, the extracted claims should be separated by comas for example like this - claim 1,claim 2,....claim n\n\n### Product OCR input:\n3 kg + 1 kg Free*\nove your\nclothes\nHenko\nSTAIN care\nBIO STAING\nPOWER**\nCare\nENZYMES\n+\n1 kg Free*\n850 \n\n ### Extracted Claims:  [/INST] \n  Care YOUR clothes, BIO STAIN Power, Care, Enzymes, +, 1 kg Free*,  850,  3 kg + 1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free*,  1 kg Free',
 '<s>[INST] \n\n###Instruction:\n    You are an expert in extracting product claims from Product OCR input of the product image, please consider the below points before extracting those claims \n    Consider the below conditions before extracting the claims: \n        Condition 1 - Extract only the English claims that the product makes and please do not provide any explanation\n        Condition 2 - Consider tests and certifications mentioned as claims\n        Condition 3 - Do not consider quantity, expiry or directions to use as claims\n        Condition 4 - Do not consider any nutritional information as claims\n        Condition 5 - Do not consider contact, address or website information\n        Condition 6 - If you do not find any product claims return None\n        Condition 7 - Ignore other language claims\n        Condition 9 - Please do not consider volume information as claim and also please do not consider instructions provided as claims \n        Condition 8 - Please do not provide any explanation, the extracted claims should be separated by comas for example like this - claim 1,claim 2,....claim n\n\n### Product OCR input:\nGENTLE & SAFE\nDetergent\nLiquid\nBlissful Petal\n*** L\nPowered by\nEnzymes\nSkin Safe Plant-based Stain Remover Colour Safe\n1L\nTM\nsuper\nguard\n100% Love, 0% Toxins\nFrans \n\n ### Extracted Claims:  [/INST] \n  GENTLE & SAFE, Powered by Enzymes, SKIN SAFE PLANT BASED STAIN REMOVER, COLOUR SAFE, 1L,  100% Love, 0% Toxins, Frans,  1L,  100% Love, 0% Toxins, Frans,  1L,  100% Love, 0% Toxins, Frans,  1L,  100% Love, 0% Toxins, Frans,  1L,  100% Love, 0% Toxins, Frans,  1L,  100% Love, 0% Toxins, Frans,  1L,  100% Love, 0% Toxins, Frans,  1L,  100% Love, 0% Toxins, Frans,  1L,  100% Love, 0% Toxins, Frans,  1L,  100% Love, 0% Toxins, Frans,  1L,  100% Love, 0% Toxins, Frans,  1L,  100% Love, 0% Toxins, Frans,  1L,  100% Love, 0% Toxins, Frans,  1L,  100% Love, 0% Toxins, Frans,  1L,  100% Love, 0% Toxins, Frans,  1L,  100% Love, 0% Toxins, Frans,  1L,  100% Love, 0% Toxins, Frans,  1L,  100% Love, 0% Toxins, Frans,  1L,  100% Love, 0% Toxins, Frans,  1L,  100% Love, 0% Toxins, Frans,  1L,  100% Love, 0% Toxins, Frans,  1L,  100% Love, 0% Toxins, Frans,  1L,  100% Love, 0% Toxins, Frans,  1L,  100% Love, 0% Toxins, Frans,  1L,  100% Love,',
 "<s>[INST] \n\n###Instruction:\n    You are an expert in extracting product claims from Product OCR input of the product image, please consider the below points before extracting those claims \n    Consider the below conditions before extracting the claims: \n        Condition 1 - Extract only the English claims that the product makes and please do not provide any explanation\n        Condition 2 - Consider tests and certifications mentioned as claims\n        Condition 3 - Do not consider quantity, expiry or directions to use as claims\n        Condition 4 - Do not consider any nutritional information as claims\n        Condition 5 - Do not consider contact, address or website information\n        Condition 6 - If you do not find any product claims return None\n        Condition 7 - Ignore other language claims\n        Condition 9 - Please do not consider volume information as claim and also please do not consider instructions provided as claims \n        Condition 8 - Please do not provide any explanation, the extracted claims should be separated by comas for example like this - claim 1,claim 2,....claim n\n\n### Product OCR input:\nPUER\nLiquid Detergent | ADVANCE CARE\nSTAIN BUSTING 2X ACTIVE\nPOWER OF 6 BIO-ENZYMES\nPLANT\nPOWER\nEVERLASTING\nBLOOM\nACTIVE\n2XLOAD \n\n ### Extracted Claims:  [/INST] \n  LIQUID detergent, Advance Care, Stain Busting 2X ACTIVE Power of 6 bio enzymes, PLANT Power, Everlasting Bloom, Active, 2XLoad,  \n\n ### Explanation:\n  LIQUID detergent is a type of detergent that is designed to be used in a washing machine. Advance Care is a term that refers to the detergent's ability to remove tough stains and leave clothes looking and feeling their best. Stain Busting 2X ACTIVE Power of 6 bio enzymes refers to the detergent's ability to break down and remove tough stains. PLANT Power refers to the detergent's natural ingredients. Everlasting Bloom refers to the detergent's ability to leave clothes smelling fresh and clean. Active refers to the detergent's ability to penetrate and clean clothes effectively. 2XLoad refers to the detergent's ability to clean clothes in half the time of other detergents.\n\n ### Comments:\n  The detergent's ability to clean clothes in half the time of other detergents is likely due to its advanced formula, which includes a blend of natural ingredients and enzymes that work together to break down and remove tough stains. The detergent's natural ingredients are likely to be gentle on clothes and the environment, making it a good choice for those who want to use a detergent that is both effective and eco-friendly.\n\n ### Recommendation:\n  This detergent is a good choice for those who want to use a detergent that is both effective and eco-friendly. It is particularly well-suited for those who have tough stains to remove, as it includes a blend of natural ingredients and enzymes that work together to break down and remove tough stains.\n\n ### Technical Specifications:\n \n ### Packaging:\n \n ### Marketed By:\n \n ### Mfg. By:\n \n ### M.R.P.:\n \n ### Net Vol.:\n \n ### MFD.:\n \n ### Exp. Date:\n \n ### Packing Date:\n \n ### Batch No.:\n \n ### Instr. To Use:\n \n ### Caution:\n \n ### Store In:\n \n ### M. of Mfg.:\n \n ### M. of M. MFD.:\n ",
 '<s>[INST] \n\n###Instruction:\n    You are an expert in extracting product claims from Product OCR input of the product image, please consider the below points before extracting those claims \n    Consider the below conditions before extracting the claims: \n        Condition 1 - Extract only the English claims that the product makes and please do not provide any explanation\n        Condition 2 - Consider tests and certifications mentioned as claims\n        Condition 3 - Do not consider quantity, expiry or directions to use as claims\n        Condition 4 - Do not consider any nutritional information as claims\n        Condition 5 - Do not consider contact, address or website information\n        Condition 6 - If you do not find any product claims return None\n        Condition 7 - Ignore other language claims\n        Condition 9 - Please do not consider volume information as claim and also please do not consider instructions provided as claims \n        Condition 8 - Please do not provide any explanation, the extracted claims should be separated by comas for example like this - claim 1,claim 2,....claim n\n\n### Product OCR input:\nNimre\nस्वच्छता भी, सेहत भी\nACTIVE\nNimrex Detergent Powder\nis a superior quality\nwashing powder, which\ncontains optical\nbrightening agent and\nsurfactant mixed in\noptimal ratio to give you\nthe ultimate in washing\nexperience in terms of\neffortless washing, long\nlasting pleasant fragrance\nand bright clothes.\nNet Weight: 1kg\nM.R.P.\n:\n(incl. of all Taxes)\nPkd. on\n65/\n: SEP. 2021\nBe Belore 36 Months From Manufacture\nMkt. & Mfd. By\nNimbus Global\nIndia Limited\nE-208, Sec-63, Noida\nGautam Budh Nagar, U.P\nE: nimbus.apcdiv@gmail.com\ner nirnbungi.com\nNimre\nस्वच्छता मी, सेहत भी\nACTIVE\nDETERGENT POWDER\nTry Our Other Products Also\nBEA\nNimbus\np\nNimbus\nNimbus\nDirection for Usage\nimrex Detergent Powder\neparate your White, Light &\nark Colour Clothes. Add\nmrex Washing Powder for\nod wash. Dissolve Powder\nmpletely before adding\nothes. For machine use add\nscoops, or bucket wash use\nscoop of Nimrex Detergent\nowder\nCAUTION:\nKeep Nimrex\nDetergent Powder\naway from children\nf this product gets\nnto the eyes by any\nchance, rinse\nhoroughly with clean\nvater. If this product\ns accidently \n\n ### Extracted Claims:  [/INST] \n  SUPERIOR QUALITY, washing powder, contains optical brightening agent AND surfactant mixed in optimal ratio, EFFORTLESS washing, long lasting PLEASANT FRAGRANCE AND bright clothes, Try Our Other Products Also,  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n',
 '<s>[INST] \n\n###Instruction:\n    You are an expert in extracting product claims from Product OCR input of the product image, please consider the below points before extracting those claims \n    Consider the below conditions before extracting the claims: \n        Condition 1 - Extract only the English claims that the product makes and please do not provide any explanation\n        Condition 2 - Consider tests and certifications mentioned as claims\n        Condition 3 - Do not consider quantity, expiry or directions to use as claims\n        Condition 4 - Do not consider any nutritional information as claims\n        Condition 5 - Do not consider contact, address or website information\n        Condition 6 - If you do not find any product claims return None\n        Condition 7 - Ignore other language claims\n        Condition 9 - Please do not consider volume information as claim and also please do not consider instructions provided as claims \n        Condition 8 - Please do not provide any explanation, the extracted claims should be separated by comas for example like this - claim 1,claim 2,....claim n\n\n### Product OCR input:\nNimre\nACTIVE\nof\nNimrex Detergent Powde\nis a superior quality\nwashing powder, which\ncontains optical\nbrightening agent and\nsurfactant mixed in\noptimal ratio to give you\nthe ultimate in washing\nexperience in terms of\neffortless washing, long\nlasting pleasant fragran\nand bright clothes.\nNet Weight: 1kg\nM.R.P.\n(incl. of all Taxes)\nPkd. on\nBest Before 36 Months From Manufacture\nSEP. 2021\nMkt. & Mfd. By\nNimbus Global\nIndia Limited\nE-208, Sec-63, Noida\nGautam Budh Nagar, U.P\nE: nimbus.apcdiv@gmail.com\nwww.nimbusgi.com\nCustomer Care No.120-4221084\nNOVOCHEM ENGINEERING INDIA LLP.\nDwarka Sector-23, New Delhi\n12\nNimre\nACTIVE\nDETERGENT POWDER\nTry Our Other Products Also\nNimbos\nNimbun\nDirection for Usage\nNimrex Detergent Powder\nSeparate your White, Light &\nDark Colour Clothes. Add\nNimrex Washing Powder for\ngood wash. Dissolve Powder\ncompletely before adding\nclothes. For machine use add\n2 scoops, or bucket wash use\n1 scoop of Nimrex Detergent\nPowder\nCAUTION\nKeep Nimrex,\nDetergent Powder\naway from children\nif this product gels\ninto the eyes by any\nchance, rinse\nthoroughly with clean\nwater. If this product\nis accidently\nswallowed, seek\nmedical advice \n\n ### Extracted Claims:  [/INST] \n  Nimrex DETERGENT powder IS A superior quality washing powder, which contains optical brightening agent AND surfactant mixed in optimal ratio to give you the ultimate in washing experience in terms of effortless washing, long lasting PLEASANT FRAGRANCE AND bright clothes., Try Our Other Products Also, Nimrex DETERGENT powder IS a detergent powder, Nimrex DETERGENT powder IS a powder, Nimrex DETERGENT powder IS a detergent, Nimrex DETERGENT powder IS a washing powder, Nimrex DETERGENT powder IS a powder for washing, Nimrex DETERGENT powder IS a powder for la',
 '<s>[INST] \n\n###Instruction:\n    You are an expert in extracting product claims from Product OCR input of the product image, please consider the below points before extracting those claims \n    Consider the below conditions before extracting the claims: \n        Condition 1 - Extract only the English claims that the product makes and please do not provide any explanation\n        Condition 2 - Consider tests and certifications mentioned as claims\n        Condition 3 - Do not consider quantity, expiry or directions to use as claims\n        Condition 4 - Do not consider any nutritional information as claims\n        Condition 5 - Do not consider contact, address or website information\n        Condition 6 - If you do not find any product claims return None\n        Condition 7 - Ignore other language claims\n        Condition 9 - Please do not consider volume information as claim and also please do not consider instructions provided as claims \n        Condition 8 - Please do not provide any explanation, the extracted claims should be separated by comas for example like this - claim 1,claim 2,....claim n\n\n### Product OCR input:\nTM\nneotea\nIssai\n12421003000523\nIngredients\nPoochi\nUrandai\nALISHAKH\nA BOARD IN\nISO\nHII\n9001:2015\nPoochi urandai\nAbout this item:\n+ Bathroom smell also use\n+ Nice fragrance\n+ Keep away from children toilet use\nProduct description\nUse for urinal basin, good for washing basin, hote,\noffice, industrial, restaurants, good fragrance ball,\nstrong ball, in this pack 3 urinal cake, good and strong\nfragrance, strong fragrance keep away from children,\npink color, keep our bathroom use for bathroom,\nnaphthalene urinal cake balls repelling or killing insects\nsuch as moths and silverfish.\nManufactured By:\nBakers spices & Ingredients Pvt ltd.\nCoimbatore, Tamil nadu, India.\nMktd by\nNeoteric DCBA Ideas,Coimbatore, TN, India, Ph:+91-9600511023\nneotea.in@gmail.com www.neotericindia.com\n100\nPacked on: 15/06/2022\nExpiry on 14/06/2023\nNet. wt\n:250gm\n:674/-\nM.R.F\nShelf life 365 days\nNO PRESERVATIVES\nONO\nSIDE EFFECTS\nfyp \n\n ### Extracted Claims:  [/INST] \n  Poochi, URANDai, A BOARD IN, ISO, HII,  9001:2015,   Keep away from children toilet use,   Use for urinal basin, good for washing basin, hote, office, industrial, restaurants, good fragrance ball, strong fragrance,   Keep away from children,   pink color,   Keep our bathroom use for bathroom,   naphthalene urinal cake balls repelling or killing insects such as moths and silverfish.,   Manufactured By: Bakers spices & Ingredients Pvt ltd. Coimbatore, Tamil nadu, India.,   Mktd by: Neoteric DCBA Ideas,Coimbatore, TN, India, Ph:+91-9600511023,   neotea.in@gmail.com www.neotericindia.com,  100,   Packed on: 15/06/2022,   Expiry on 14/',
 '<s>[INST] \n\n###Instruction:\n    You are an expert in extracting product claims from Product OCR input of the product image, please consider the below points before extracting those claims \n    Consider the below conditions before extracting the claims: \n        Condition 1 - Extract only the English claims that the product makes and please do not provide any explanation\n        Condition 2 - Consider tests and certifications mentioned as claims\n        Condition 3 - Do not consider quantity, expiry or directions to use as claims\n        Condition 4 - Do not consider any nutritional information as claims\n        Condition 5 - Do not consider contact, address or website information\n        Condition 6 - If you do not find any product claims return None\n        Condition 7 - Ignore other language claims\n        Condition 9 - Please do not consider volume information as claim and also please do not consider instructions provided as claims \n        Condition 8 - Please do not provide any explanation, the extracted claims should be separated by comas for example like this - claim 1,claim 2,....claim n\n\n### Product OCR input:\nFast\nNEW\nDissolution\nQuick\nCleaning\nसफेद\nडिटरजेंट पाउडर\nHigh Foam\n515\nSoft\nManufacturerkeled by:\nShantinath et argents Pvt Lt\nby, rehcharacters the code embossed on\nfor unique blend\n89360224901207\nCUSTOMER CARE\n3 2248 9408\nthe box fo low O. A) 16 Road, Kolkata 700058,\nBF BT Row Godown\nKolkata 700058, C) Plot\n60 Bahmapudustrial Park, Village - Sila,\na Guwa Ask\non Hands\nterge.com OTHER\nGeneration\n#MRP. ?\nBA)08/22 130/-/65/Kg)\nNet Wt.\n2 kg\nPkd.\nSafed with its\nnext-generation SmartZymes Technology,\nprovides Smart Care & Enduring Freshness\nfor your clothes.\nEnduring\nFreshness\nLong-lasting\nRefreshing\nFragrance.\nSmart\nvmes\nInstructions For Use\nFor hand wash\n1 scoop\nFor machine wash\n2 scoops\nSmart Care\nRemarkably Clean\n& Vibrant Clothes\nscoop not included. \n\n ### Extracted Claims:  [/INST] \n  Smart Care, Enduring Freshness, Long Lasting Refreshing FRAGRANCE, Smart VMES, Remarkably CLEAN & VIBRANT clothes, 515, High FOAM, Soft, 89360224901207, Customer Care,  16 Road, Kolkata 700058, BF BT Row Godown, Kolkata 700058, Plot 60 Bahmapudustrial Park, Village - Sila, Guwa Ask, on Hands,  #MRP.?, 2 kg, PKD, SAFED with its next generation SmartZymes Technology, provides Smart Care & Enduring Freshness for your clothes., Smart Care, Remarkably CLEAN & VIBRANT clothes, Smart VMES, 515, High FOAM, Soft, 89360224901207, Customer Care,  16 Road, Kolkata 700058, BF BT Row Godown, Kolkata 700058, Plot 60 Bahmapudustrial Park, Village']

In [41]:
len(response_llma)

22

In [42]:
test_complete["preidction_llma_for_one_epoch"]=response_llma

In [44]:
test_complete.to_csv("/LLM_results/Zephyr_LLM_res.csv",index=False)

In [1]:
import pandas as pd

In [25]:
tmp=pd.read_csv("/LLM_results/Zephyr_LLM_res.csv")
tmp.head(2)

,id,Image_side,claim_0,claim_1,claim_2,claim_3,claim_4,claim_5,claim_6,claim_7,...,claim_16,claim_17,claim_18,claim_19,claim_20,ocr_info,prompt_cliam,text,prediction_zephyr,preidction_llma_for_one_epoch
0,9995458_0,Front,LIQUID detergent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,5\nMizzen\nLIQUID DETERGENT\nDirection For Use...,"LIQUID detergent,",5\nMizzen\nLIQUID DETERGENT\nDirection For Use...,<|system|>\n ###Instruction:\n You are an e...,<s>[INST] \n\n###Instruction:\n You are an ...
1,9995460_0,Front,easy & Clear wash LIQUID,Quick drying,removing TOUGH STAINS,Fresh SCENT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,DMAXX\nEasy & Clear Wash Liquid\nLIQUID\nDETER...,"easy & Clear wash LIQUID, Quick drying, remov...",DMAXX\nEasy & Clear Wash Liquid\nLIQUID\nDETER...,<|system|>\n ###Instruction:\n You are an e...,<s>[INST] \n\n###Instruction:\n You are an ...


In [161]:
row_nuw=12

In [162]:
print(tmp["ocr_info"][row_nuw])

00
罪
Pfronel
Silk & Chiffon
LIQUID DETERGENT
Gently clean, softnes & fluffy
Retain fabric strengthen & natural colour
Specially formulated for silk fabric
5L/169 US fl.oz.


In [163]:
print(tmp["prompt_cliam"][row_nuw])

 GENTLY CLEAN, softness & fluffy, retain FABRIC strength & NATURAL COLOUR, Specially formulated FOR Silk FABRIC,


In [164]:
print(tmp["prediction_zephyr"][row_nuw])

<|system|>
 ###Instruction:
    You are an expert in extracting product claims from Product OCR input of the product image, please consider the below points before extracting those claims 
    Consider the below conditions before extracting the claims: 
        Condition 1 - Extract only the English claims that the product makes and please do not provide any explanation
        Condition 2 - Consider tests and certifications mentioned as claims
        Condition 3 - Do not consider quantity, expiry or directions to use as claims
        Condition 4 - Do not consider any nutritional information as claims
        Condition 5 - Do not consider contact, address or website information
        Condition 6 - If you do not find any product claims return None
        Condition 7 - Ignore other language claims
        Condition 9 - Please do not consider volume information as claim and also please do not consider instructions provided as claims 
        Condition 8 - Please do not provide any ex

In [165]:
print(tmp["preidction_llma_for_one_epoch"][row_nuw])

<s>[INST] 

###Instruction:
    You are an expert in extracting product claims from Product OCR input of the product image, please consider the below points before extracting those claims 
    Consider the below conditions before extracting the claims: 
        Condition 1 - Extract only the English claims that the product makes and please do not provide any explanation
        Condition 2 - Consider tests and certifications mentioned as claims
        Condition 3 - Do not consider quantity, expiry or directions to use as claims
        Condition 4 - Do not consider any nutritional information as claims
        Condition 5 - Do not consider contact, address or website information
        Condition 6 - If you do not find any product claims return None
        Condition 7 - Ignore other language claims
        Condition 9 - Please do not consider volume information as claim and also please do not consider instructions provided as claims 
        Condition 8 - Please do not provide any ex

In [166]:
tmp_2=pd.read_csv("/LLM_results/Mistral_LLM_res.csv")


In [167]:
tmp_2.head(1)

,id,Image_side,claim_0,claim_1,claim_2,claim_3,claim_4,claim_5,claim_6,claim_7,...,claim_15,claim_16,claim_17,claim_18,claim_19,claim_20,ocr_info,prompt_cliam,text,prediction_Mistral
0,9995458_0,Front,LIQUID detergent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,###Instruction:\n You are an expert in extr...,"LIQUID detergent,",###Instruction:\n You are an expert in extr...,<s> ###Instruction:\n You are an expert in ...


In [168]:
print(tmp_2["prediction_Mistral"][row_nuw])

<s> ###Instruction:
    You are an expert in extracting product claims from Product OCR input of the product image, please consider the below points before extracting those claims 
    Consider the below conditions before extracting the claims: 
        Condition 1 - Extract only the English claims that the product makes and please do not provide any explanation
        Condition 2 - Consider tests and certifications mentioned as claims
        Condition 3 - Do not consider quantity, expiry or directions to use as claims
        Condition 4 - Do not consider any nutritional information as claims
        Condition 5 - Do not consider contact, address or website information
        Condition 6 - If you do not find any product claims return None
        Condition 7 - Ignore other language claims
        Condition 9 - Please do not consider volume information as claim and also please do not consider instructions provided as claims 
        Condition 8 - Please do not provide any explanatio

In [169]:
tmp_3=pd.read_csv("/LLM_results/phi_res_new.csv")


In [170]:
tmp_3.head(1)

,id,Image_side,claim_0,claim_1,claim_2,claim_3,claim_4,claim_5,claim_6,claim_7,...,claim_14,claim_15,claim_16,claim_17,claim_18,claim_19,claim_20,ocr_info,prompt_cliam,phi_res
0,9995458_0,Front,LIQUID detergent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5\nMizzen\nLIQUID DETERGENT\nDirection For Use...,"LIQUID detergent,,,,,,,,,,,,,,,,,,,,",['###Instruction:\n\n\nYou are an expert in ex...


In [171]:
from ast import literal_eval


In [172]:
print((literal_eval(tmp_3["phi_res"][row_nuw]))[0])

###Instruction:


You are an expert in extracting product claims from Product OCR input of the product image, please consider the below points before extracting those claims
Consider the below conditions before extracting the claims:
Condition 1 - Extract only the English claims that the product makes and please do not provide any explanation
Condition 2 - Consider tests and certifications mentioned as claims
Condition 3 - Do not consider quantity, expiry or directions to use as claims
Condition 4 - Do not consider any nutritional information as claims
Condition 5 - Do not consider contact, address or website information
Condition 6 - If you do not find any product claims return None
Condition 7 - Ignore other language claims
Condition 9 - Please do not consider volume information as claim and also please do not consider instructions provided as claims
Condition 8 - Please do not provide any explanation, the extracted claims should be separated by comas for example like this - claim 1,